In [ ]:
from garminconnect import (
    Garmin,
    GarminConnectConnectionError,
    GarminConnectTooManyRequestsError,
    GarminConnectAuthenticationError,
)
import requests
from datetime import date
import numpy as np

today = date.today()

### Get email and password

In [ ]:
USER_EMAIL, USER_PASSWORD = np.loadtxt("./garmin_user_pass.txt",dtype=str)

### Login script

In [ ]:
try:
    client = Garmin(USER_EMAIL, USER_PASSWORD)
    client.login()
    print(f"logged into Garmin as",client.get_full_name())

except (
    GarminConnectConnectionError,
    GarminConnectAuthenticationError,
    GarminConnectTooManyRequestsError,
) as err:
    print("Error occurred during Garmin Connect Client init: %s" % err)

except Exception as e:  # pylint: disable=broad-except
    print("Unknown error occurred during Garmin Connect Client init")
    print(e)

### Get recent stats

In [ ]:
raw_stats = client.get_stats(today.isoformat())
sleep_stats = client.get_sleep_data(today.isoformat())
stats_to_get = ['bodyBatteryMostRecentValue', 
                'stressQualifier', 
                'restingHeartRate', 
                'lastSevenDaysAvgRestingHeartRate',
                'measurableAwakeDuration', 
                'measurableAsleepDuration']
sleep_stats_to_get = ['deepSleepSeconds', 
                      'lightSleepSeconds',
                      'remSleepSeconds']
stats = []
for s in stats_to_get:
    stats.append(raw_stats[s])
for s in sleep_stats_to_get:
    stats.append(sleep_stats['dailySleepDTO'][s]) #outputs seconds spent in each sleep state
    
print(stats)

### Suggestions for some slack status updates

In [ ]:
hr, stress_state, resting_hr, resting_hr_average, awake_time, asleep_time, deep_sleep, light_sleep, rem_sleep = stats


if resting_hr > resting_hr_average:
    '''
    I tend to find that my resting heart rate going up tends to correlate with my energy
    so if I do something strenuous the day before I generally find that my resting hr is
    higher the next day and I feel more tired
    '''
    print("Status: Feeling a bit tired today")
    
if ((awake_time/3600.0) > 10):
    '''
    If you have been awake for more than 10 hours its probably starting to get late in the
    day for you
    '''
    print("Status: Been awake for a while now")
    
if(stress_state == "CALM"):
    '''
    Stress on Garmin is to do with heart rate variability (I think)
    '''
    print("Status: Ready to focus")